In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

In [ ]:
# Pandas is used for data manipulation and analysis
import pandas as pd 

# Numpy is used for large, multi-dimensional arrays and matrices, along with mathematical operators on these arrays
import numpy as np

from google.colab import files
import time

In [ ]:
os.chdir('/content/drive/MyDrive/capstone/')

In [ ]:
! pip install fasttext

In [ ]:
import fasttext

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/capstone/Final_Articles_Data_25June.csv', engine="python")

In [ ]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Article URL,Headline,Article Text,Sub Domain,Domain,Author,Cleaned_Article,Cleaned_Headline,Cleaned_Author,Word_Count,Article_Size
0,0,0.0,https://www.entrepreneur.com/article/370874,\n\t\t\t\tIs Launching a New Brand the Right M...,\n\n\n\t\t\t\t\t\t\t\tJune\n\t\t\t\t\t\t\t\t10...,Branding,Marketing,Melissa Packham,One of the most common challenges I see client...,Is Launching a New Brand the Right Move for Yo...,Melissa Packham,1013,Large
1,1,1.0,https://www.entrepreneur.com/article/372254,\n\t\t\t\t3 Reasons Simple Isn't Always Better...,\n\n\n\t\t\t\t\t\t\t\tJune\n\t\t\t\t\t\t\t\t10...,Branding,Marketing,Zaheer Dodhia,"Lets say that, like me, youve been thinking ab...",3 Reasons Simple Isn't Always Better When It C...,Zaheer Dodhia,882,Medium
2,2,2.0,https://www.entrepreneur.com/article/368356,\n\t\t\t\tFrom Idea to Revenue: A Six-Step For...,\n\n\n\t\t\t\t\t\t\t\tJune\n\t\t\t\t\t\t\t\t10...,Launching a Business,Marketing,Jessica O'Connell,"If youre like most entrepreneurs, you may have...",From Idea to Revenue A Six-Step Formula to Lau...,Jessica O'Connell,1409,Large
3,3,3.0,https://www.entrepreneur.com/article/373825,\n\t\t\t\tAvoid This Common Mistake When Writi...,\n\n\n\t\t\t\t\t\t\t\tJune\n\t\t\t\t\t\t\t\t10...,Writing a Book,Marketing,R. Paulo Delgado,I had an interesting project come across my de...,Avoid This Common Mistake When Writing Your Fi...,R. Paulo Delgado,952,Medium
4,4,4.0,https://www.entrepreneur.com/article/372679,\n\t\t\t\tIs it Time to Say RIP to RFPs?\n\t\t\t,"\n\n\n\t\t\t\t\t\t\t\tJune\n\t\t\t\t\t\t\t\t9,...",Entrepreneurs,Marketing,Heather Ripley,"In a world where personal touch matters, why a...",Is it Time to Say RIP to RFPs,Heather Ripley,723,Medium


In [ ]:
data = df[['Cleaned_Article','Domain']].copy()
data.rename(columns ={'Cleaned_Article' : 'Cleaned_Article_Text'}, inplace = True)

In [ ]:
data.head()

,Cleaned_Article_Text,Domain
0,One of the most common challenges I see client...,Marketing
1,"Lets say that, like me, youve been thinking ab...",Marketing
2,"If youre like most entrepreneurs, you may have...",Marketing
3,I had an interesting project come across my de...,Marketing
4,"In a world where personal touch matters, why a...",Marketing


In [ ]:
data.describe()

,Cleaned_Article_Text,Domain
count,29631,29631
unique,27769,5
top,"As far as backstory, the title really says it ...",Marketing
freq,3,7467


In [ ]:
# Shuffle training dataframe
data_shuffled = data.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
data_shuffled.head()

,Cleaned_Article_Text,Domain
762,The following excerpt is from Dan S. Kennedys ...,Marketing
26694,"You're reading Entrepreneur Middle East, an in...",Technology
19411,"In the midst of a global health crisis, it can...",Strategy
12865,"Making decisions, even small, seemly harmless ...",Leadership
4702,"JDate, the premier community for Jewish single...",Marketing


### Referrd Fast Text dcoumentation example  https://fasttext.cc/docs/en/supervised-tutorial.html

In [ ]:
help(fasttext.FastText)

Help on module fasttext.FastText in fasttext:

NAME
    fasttext.FastText

DESCRIPTION
    # Copyright (c) 2017-present, Facebook, Inc.
    # All rights reserved.
    #
    # This source code is licensed under the MIT license found in the
    # LICENSE file in the root directory of this source tree.

FUNCTIONS
    cbow(*kargs, **kwargs)
    
    eprint(*args, **kwargs)
    
    load_model(path)
        Load a model given a filepath and return a model object.
    
    read_args(arg_list, arg_dict, arg_names, default_values)
    
    skipgram(*kargs, **kwargs)
    
    supervised(*kargs, **kwargs)
    
    tokenize(text)
        Given a string of text, tokenize it and return a list of tokens
    
    train_supervised(*kargs, **kwargs)
        Train a supervised model and return a model object.
        
        input must be a filepath. The input text does not need to be tokenized
        as per the tokenize function, but it must be preprocessed and encoded
        as UTF-8. You might wan

In [ ]:
from io import StringIO
import csv

col = ['Domain', 'Cleaned_Article_Text']

data_shuffled = data_shuffled[col]
data_shuffled['Domain']=['__label__'+ s for s in data_shuffled['Domain']]
data_shuffled['Cleaned_Article_Text']= data_shuffled['Cleaned_Article_Text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
data_shuffled.to_csv(r'/content/drive/MyDrive/capstone/art.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [ ]:
#Train test split - note the file names without extension but with .train and .test
!head -n 26668 "/content/drive/MyDrive/capstone/art.txt" > "/content/drive/MyDrive/capstone/art.train"
!tail -n 2963 "/content/drive/MyDrive/capstone/art.txt" > "/content/drive/MyDrive/capstone/art.test"

In [ ]:
#train 
# default lr =0.1, epoc =5,
model = fasttext.train_supervised(input='art.train')

In [ ]:
#this saves the model inside "fastText" folder
model.save_model("model_fast_base.bin") 

In [ ]:
#testß
model.test("/content/drive/MyDrive/capstone/art.test")

(2963, 0.6871414107323659, 0.6871414107323659)

In [ ]:
# 2963 test records, Precision 68.71 Recall 68.71%

In [ ]:
#lr = learning rate n range from 0.1 to 1

# lrUpdateRate similar to batch size

#wordNgrams = 3 means Trigram. improves model performance by using sequential three words

#bucket = Corresponds to the total size of array allocated for all the ngram tokens. It is set via the -bucket flag, and is set to be 2000000 by default.
#The size of the model will increase linearly with the number of buckets. The size of the input matrix is DIM x (VS + BS), where VS is the number of words in the vocabulary and BS is the number of buckets

#dim = size of word vectors.100 is default

#loss = 'hs',The hierarchical softmax, instead of the regular softmax make the training faster

In [ ]:
#Model 2

The hierarchical softmax, instead of the regular softmax make the training faster.This can be done with the option -loss hs:

In [ ]:
model2 = fasttext.train_supervised(input="art.train", lr=1.0, epoch=25, wordNgrams=3, bucket=200000, dim=300, loss='hs') #32 mins


Dim represents the dimension of the hidden layer in training, and thus the dimension of the embeddings, and is set via the -dim flag. This is set to 100 by default.

In [ ]:
model2.save_model("model_fast_improved2.bin") 

In [ ]:
model2.test("/content/drive/MyDrive/capstone/art.test")

(2963, 0.7057036787040162, 0.7057036787040162)

In [ ]:
model3 = fasttext.train_supervised(input="/content/drive/MyDrive/capstone/art.train", lr=0.9, epoch=25, wordNgrams=3, bucket=200000, dim=100, loss='ova') #17 mins
#loss = 'ova',independent classifers for each class in case of multiple class

In [ ]:
model3.save_model("model_fast_improved1.bin") 

In [ ]:
model3.test("/content/drive/MyDrive/capstone/art.test")

(2963, 0.7195410057374283, 0.7195410057374283)

In [ ]:
#Reference

In [ ]:
# cleaning the data and fed back to model #https://www.kaggle.com/theoviel/improve-your-score-with-text-preprocessing-v2
#http://ethen8181.github.io/machine-learning/deep_learning/multi_label/fasttext.html
#https://fasttext.cc/docs/en/supervised-tutorial.html